# CS211: Data Privacy
## In-Class Exercise, week of 10/31/2022

In [5]:
# Load the data and libraries
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')

def laplace_mech(v, sensitivity, epsilon):
    return v + np.random.laplace(loc=0, scale=sensitivity / epsilon)

def gaussian_mech(v, sensitivity, epsilon, delta):
    return v + np.random.normal(loc=0, scale=sensitivity * np.sqrt(2*np.log(1.25/delta)) / epsilon)

def gaussian_mech_vec(vec, sensitivity, epsilon, delta):
    return [v + np.random.normal(loc=0, scale=sensitivity * np.sqrt(2*np.log(1.25/delta)) / epsilon)
            for v in vec]

def pct_error(orig, priv):
    return np.abs(orig - priv)/orig * 100.0

# adult = pd.read_csv('https://github.com/jnear/cs211-data-privacy/raw/master/homework/adult_with_pii.csv')

/tmp/ipykernel_2801/3054158115.py:6: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-whitegrid')


In [6]:
# Load data files
import numpy as np
import urllib.request
import io

url_x = 'https://github.com/jnear/cs211-data-privacy/raw/master/slides/adult_processed_x.npy'
url_y = 'https://github.com/jnear/cs211-data-privacy/raw/master/slides/adult_processed_y.npy'

with urllib.request.urlopen(url_x) as url:
    f = io.BytesIO(url.read())
X = np.load(f)

with urllib.request.urlopen(url_y) as url:
    f = io.BytesIO(url.read())
y = np.load(f)

In [7]:
# X is the same as adult.csv but setup nicley for machine learning
# Everything is a number, categorical values are one-hot encoded 
#     To one hot encode something, create an n length vector, where n is the # of possible values 
#     Set all to 0 except the one for the selected
# One hot encoding expands the # of coumns 19 -> 104

In [8]:
# Split data into training and test sets
training_size = int(X.shape[0] * 0.8)

x_train = X[:training_size]
x_test = X[training_size:]

y_train = y[:training_size]
y_test = y[training_size:]

print('Train and test set sizes:', len(y_train), len(y_test))

Train and test set sizes: 36176 9044


## Question 1

Using scikit-learn, train a logistic regression model on the training data loaded above.

In [9]:
from sklearn.linear_model import LogisticRegression

In [10]:
def train_model():
    model = LogisticRegression().fit(x_train, y_train)
    return model

model = train_model()
print('Model coefficients:', model.coef_[0])
print('Model accuracy:', np.sum(model.predict(X_test) == y_test)/X_test.shape[0])

Model coefficients: [ 4.78139205e-01 -1.79956514e-01 -3.40239314e-02  6.84210447e-02
 -5.91027836e-01 -3.66111064e-01 -1.06083247e+00 -6.69749843e-01
 -6.36897045e-01 -4.60622052e-01 -5.10623615e-01 -3.97837711e-01
 -7.57652238e-01 -7.81287024e-01  6.35764981e-02  1.18067100e-01
  5.10746634e-01  1.00307693e+00 -1.12284805e-01  7.38198230e-01
 -1.18443431e+00  1.28199985e+00  1.10331842e-01 -8.97901491e-01
  1.50203219e+00  1.25193383e+00 -5.83474111e-01 -1.31380633e+00
 -8.89748113e-01 -7.54427536e-01 -5.67454198e-02  5.14477437e-02
 -8.14655572e-04  7.21850375e-01 -8.96540282e-01 -6.94306000e-01
 -3.23897413e-01 -9.41104911e-01 -1.09691775e+00  4.72866034e-01
  4.76951082e-01  2.21548702e-01  5.09549054e-01 -1.29278122e-01
 -3.74278495e-01  1.79089784e-03 -7.74204436e-01 -1.05097080e+00
 -1.90670650e-01  7.02941924e-01 -5.02470396e-01 -4.51253423e-02
 -4.88120092e-01 -4.18069074e-01 -2.31606658e-01 -1.17528743e+00
 -5.10104136e-01  8.94575562e-01  6.12555630e-01 -2.84252476e-01
 -1.2

/home/semaj/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


## Question 2

Implement the *average gradient* of the loss below.

In [11]:
# The loss function measures how good our model is. The training goal is to minimize the loss.
# This is the logistic loss function.
def loss(theta, xi, yi):
    exponent = - yi * (xi.dot(theta))
    return np.log(1 + np.exp(exponent))

# This is the gradient of the logistic loss
# The gradient is a vector that indicates the rate of change of the loss in each direction
def gradient(theta, xi, yi):
    exponent = yi * (xi.dot(theta))
    return - (yi*xi) / (1+np.exp(exponent))

In [19]:
def avg_grad(theta, x, y):
    gradients = [gradient(theta, xi, yi) for xi, yi in zip(x, y)]
    return np.mean(gradients, axis=0)

## Question 3

Use the average gradient from above to implement a gradient descent algorithm.

In [24]:
def gradient_descent(iterations):
    # start out iwt theta = all zeros
    theta = np.zeros(x_train.shape[1])
    eta = 1
    # for `iterations` iterations, update theta by adding the negation for the gradient
    for _ in range(iterations):
        grads = [gradient(theta, x, y) for x, y in zip(x_train, y_train)]
        clipped_grads = [L2_clip(g, 1) for g in grads]
        grad_sum = np.sum(clipped_grads, axis=0)
        # this is the gradient update  rule 
        noisy_grad = laplace_mech_vec(grad, 1, epsilon)
        theta = theta - eta * noisy_grad
    return theta
        
theta = gradient_descent(10)
theta

NameError: name 'L2_clip' is not defined

In [25]:
# Prediction: take a model (theta) and a single example (xi) and return its predicted label
def predict(xi, theta, bias=0):
    label = np.sign(xi @ theta + bias)
    return label

def accuracy(theta):
    return np.sum(predict(X_test, theta) == y_test)/X_test.shape[0]

accuracy(theta)

0.7787483414418399

## Question 4

Implement a *noisy gradient descent* algorithm.

1. Calculate gradients for each example
2. Clip the gradients to have bounded $L2$ norm
3. Sum the clipped gradients
4. Use the Gaussian mechanism to add noise to the sum of gradients

In [ ]:
def L2_clip(v, b):
    norm = np.linalg.norm(v, ord=2)
    
    if norm > b:
        return b * (v / norm)
    else:
        return v

def noisy_gradient_descent(iterations, epsilon, delta):
    # YOUR CODE HERE
    raise NotImplementedError()

theta = noisy_gradient_descent(10, 1.0, 1e-5)
print('Final accuracy:', accuracy(theta))

In [ ]:
# TEST CASE

assert accuracy(noisy_gradient_descent(5, 0.001, 1e-5)) < 0.76
assert accuracy(noisy_gradient_descent(5, 1.0, 1e-5)) > 0.70

## Question 5

What is the *total privacy cost* of the noisy gradient descent algorithm above, and why? Argue informally that the algorithm satisfies this privacy cost. Use sequential composition.

YOUR ANSWER HERE

## Question 6

Repeat the above, but using advanced composition.

YOUR ANSWER HERE

## Question 7

Implement a version of noisy gradient descent that satisfies a *total* of $(\epsilon, \delta)$-differential privacy. Use sequential composition.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# TEST CASE

assert accuracy(noisy_gradient_descent(5, 0.001, 1e-5)) < 0.76
assert accuracy(noisy_gradient_descent(5, 1.0, 1e-5)) > 0.70